<a href="https://colab.research.google.com/github/eric-shaker/BIOENG_1340_AUTO_TUMOR_SEGMENTATION/blob/main/Tumor_Segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**This notebook contains code that trains the 3D U-Net model, saves the model, creates automated segmentations for the training set and saves them, saves the Dice Loss information, and creates the automated segmentations for the test set and saves them.**

In [ ]:
#title Mount Drive
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


#Install Packages

In [ ]:
#title Load the Prerequsite Libraries
!pip install SimpleITK
#!pip install itk

#!pip install 'simpleitk==2.0.0' --force-reinstall #previous library

import SimpleITK as sitk
import numpy as np
import os 
#import itk
!pip install torch==1.9 #currently fixes the GPU OOM error / consider removing ==1.9 later
import time
import torch 
import torch.nn as nn
print(torch.__version__)
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import zipfile
import glob

#PyTorch

!git clone https://github.com/kilgore92/PyTorch-UNet.git

%cd PyTorch-UNet
%cd src
from unet.model import UNet
%cd ..
%cd ..

#normalize function
def normalize (image):
    MIN_BOUND = np.min(image)
    MAX_BOUND = np.max(image)
    image = (image - MIN_BOUND).astype(float)/(MAX_BOUND - MIN_BOUND)
    image[image>1] = 1.
    image[image<0] = 0.
    return image

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 52.8 MB 268 kB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 831.4 MB 14 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.12.1+cu113
    Uninstalling torch-1.12.1+cu113:
      Successfully uninstalled torch-1.12.1+cu113
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.13.1+cu113 requires torch==1.12.1, but you have torch 1.9.0 which is incompatible.
torchtext 0.13.1 requires torch==1.12.1, but you have torch 1.9.0 which is incompatible.
torchaudio 0.12.1+cu113 requires torch==1.12.1, but you have torch 1.9.0 which is incompatible.
1.9.0+cu102
Cloning into 'PyTorch-UNet'...
remote: Enumerati

#U-Net

In [ ]:
#title Load Dataset Classes
class Dataset(torch.utils.data.Dataset):
  def __init__(self, orig, segment):
    self.orig = orig
    self.segment = segment

  def __len__(self):
    return len(self.orig)
  
  def __getitem__(self, index):
    image = (sitk.ReadImage(self.orig[index]))
    image = sitk.GetArrayFromImage(image)
    image = torch.from_numpy(normalize(image)).type(torch.float32)
    image = torch.reshape(image, shape=(1, 16, 16, 16))

    mask = (sitk.ReadImage(self.segment[index]))
    mask = sitk.GetArrayFromImage(mask)
    mask = np.float32(mask) #does not work with uint16
    mask = torch.from_numpy(mask).type(torch.float32)
    mask = torch.reshape(mask, shape=(1, 16, 16, 16))

    return image, mask  

# Debug mode
debug_mode = "No" #@param ["Yes", "No"] {type:"string"}
if debug_mode == "Yes":
  debug_flag = True
elif debug_mode == "No":
  debug_flag = False

#Data Loader for training 
root_path = "/content/gdrive/My Drive/BIOENG_1340/BIOENG 1340 - Final Project/images/train_images16" #@param {type: 'string'}
root_segmented = "/content/gdrive/My Drive/BIOENG_1340/BIOENG 1340 - Final Project/segs/train_segs_WT16" #@param {type: 'string'}
mainpaths = sorted(os.listdir(root_path))
maskpaths = sorted(os.listdir(root_segmented))

if debug_flag:
  mainpaths = mainpaths[:7]
  maskpaths = maskpaths[:7]


#print the segmentation files
print("Dataset Filenames")
for i in range(len(mainpaths)):
  #main files and binary masks
  print("Interval: " + str(i) + " Main: "+mainpaths[i] +" Binary Mask: "+ maskpaths[i])

Dataset Filenames
Interval: 0 Main: BraTS20_Training_007_flair16.nii.gz Binary Mask: BraTS20_Training_007_seg_WT16.nii.gz
Interval: 1 Main: BraTS20_Training_015_flair16.nii.gz Binary Mask: BraTS20_Training_015_seg_WT16.nii.gz
Interval: 2 Main: BraTS20_Training_022_flair16.nii.gz Binary Mask: BraTS20_Training_022_seg_WT16.nii.gz
Interval: 3 Main: BraTS20_Training_033_flair16.nii.gz Binary Mask: BraTS20_Training_033_seg_WT16.nii.gz
Interval: 4 Main: BraTS20_Training_036_flair16.nii.gz Binary Mask: BraTS20_Training_036_seg_WT16.nii.gz
Interval: 5 Main: BraTS20_Training_042_flair16.nii.gz Binary Mask: BraTS20_Training_042_seg_WT16.nii.gz
Interval: 6 Main: BraTS20_Training_049_flair16.nii.gz Binary Mask: BraTS20_Training_049_seg_WT16.nii.gz
Interval: 7 Main: BraTS20_Training_052_flair16.nii.gz Binary Mask: BraTS20_Training_052_seg_WT16.nii.gz
Interval: 8 Main: BraTS20_Training_053_flair16.nii.gz Binary Mask: BraTS20_Training_053_seg_WT16.nii.gz
Interval: 9 Main: BraTS20_Training_054_flair16

In [ ]:
#set training parameters in the variable partition

#seperates the training data and testing data
partition = {'orig': [os.path.join(root_path,x) for x in sorted(mainpaths)] , 
             'segment': [os.path.join(root_segmented,x) for x in sorted(maskpaths)]} 
learning_rate = 0.001#@param {type: 'number'}
num_classes = 1
batch_size = 1#@param {type: 'number'}
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

train_loader = DataLoader(dataset=Dataset(partition['orig'], partition['segment']), 
                          batch_size = batch_size, shuffle = True)

#check what is being used for testing
print("Training Data: ")
partition

cuda:0
Training Data: 


{'orig': ['/content/gdrive/My Drive/BIOENG_1340/BIOENG 1340 - Final Project/images/train_images16/BraTS20_Training_007_flair16.nii.gz',
  '/content/gdrive/My Drive/BIOENG_1340/BIOENG 1340 - Final Project/images/train_images16/BraTS20_Training_015_flair16.nii.gz',
  '/content/gdrive/My Drive/BIOENG_1340/BIOENG 1340 - Final Project/images/train_images16/BraTS20_Training_022_flair16.nii.gz',
  '/content/gdrive/My Drive/BIOENG_1340/BIOENG 1340 - Final Project/images/train_images16/BraTS20_Training_033_flair16.nii.gz',
  '/content/gdrive/My Drive/BIOENG_1340/BIOENG 1340 - Final Project/images/train_images16/BraTS20_Training_036_flair16.nii.gz',
  '/content/gdrive/My Drive/BIOENG_1340/BIOENG 1340 - Final Project/images/train_images16/BraTS20_Training_042_flair16.nii.gz',
  '/content/gdrive/My Drive/BIOENG_1340/BIOENG 1340 - Final Project/images/train_images16/BraTS20_Training_049_flair16.nii.gz',
  '/content/gdrive/My Drive/BIOENG_1340/BIOENG 1340 - Final Project/images/train_images16/BraTS2

In [ ]:
#load the 3D U-Net Model
model = UNet(n_channels=1,
             mode='3D',
             num_classes=1,
             use_pooling=False,
             )

In [ ]:
#set the Dice Loss for model learning
from keras import backend as K
from tensorflow.compat.v1 import enable_eager_execution
import tensorflow as tf
import numpy as np
import os

#Dice Loss used for criterion
import torch
import torch.nn as nn
import torch.functional as f
import numpy as np


class DICELossMultiClass(nn.Module):

    def __init__(self):
        super(DICELossMultiClass, self).__init__()

    def forward(self, output, mask):
        num_classes = output.size(1)
        dice_eso = 0
        for i in range(num_classes):
            probs = torch.squeeze(output[:, i, :, :], 1)
            mask = torch.squeeze(mask[:, i, :, :], 1)

            num = probs * mask
            num = torch.sum(num, 2)
            num = torch.sum(num, 1)

            # print( num )

            den1 = probs * probs
            # print(den1.size())
            den1 = torch.sum(den1, 2)
            den1 = torch.sum(den1, 1)

            # print(den1.size())

            den2 = mask * mask
            # print(den2.size())
            den2 = torch.sum(den2, 2)
            den2 = torch.sum(den2, 1)

            # print(den2.size())
            eps = 0.0000001
            dice = 2 * ((num + eps) / (den1 + den2 + eps))
            # dice_eso = dice[:, 1:]
            dice_eso += dice

        loss = 1 - torch.sum(dice_eso) / dice_eso.size(0)
        return loss

select_DiceLoss = "New"#@param ["Old", "New"] {type:"string"}
if select_DiceLoss == "Old":
  print("Selected: ", select_DiceLoss)
  class DICELoss(nn.Module):

      def __init__(self):
          super(DICELoss, self).__init__()

      def forward(self, output, mask):

          probs = torch.squeeze(output, 1)
          mask = torch.squeeze(mask, 1)

          intersection = probs * mask
          intersection = torch.sum(intersection, 2)
          intersection = torch.sum(intersection, 1)

          den1 = probs * probs
          den1 = torch.sum(den1, 2)
          den1 = torch.sum(den1, 1)

          den2 = mask * mask
          den2 = torch.sum(den2, 2)
          den2 = torch.sum(den2, 1)

          eps = 1e-8
          dice = 2 * ((intersection + eps) / (den1 + den2 + eps))
          # dice_eso = dice[:, 1:]
          dice_eso = dice

          loss = 1 - torch.sum(dice_eso) / dice_eso.size(0)
          return loss

if select_DiceLoss == "New":
#normalized Dice Loss code
  print("Selected: ", select_DiceLoss)
  class DICELoss(nn.Module):
      def __init__(self, weight=None, size_average=True):
          super(DICELoss, self).__init__()

      def forward(self, output, mask, eps=1e-8):
          """
          The output layer of Kilgore's 3D-UNet implementation is a LeakyReLU.
          We need to apply a sigmoid activation to the model output in order to
          constrain the target value between 0 and 1.
          See source code (Line 243):
          https://github.com/kilgore92/PyTorch-UNet/blob/master/src/unet/blocks.py#L243
          """

          output = torch.sigmoid(output)
          
          output = output.view(-1)
          mask = mask.view(-1)
          
          intersection = (output * mask).sum()                            
          dice_coeff = (2.*intersection + eps)/(output.sum() + mask.sum() + eps)  

          dice_loss = 1 - dice_coeff

          return dice_loss

Selected:  New


In [ ]:
#load a pretrained model (DON'T RUN UNLESS MODEL EXISTS)
pretrained_model_path = "/content/gdrive/MyDrive/BIOENG 1340 - Final Project/code/models/Unet_Batch1_e50_TS/Unet_Batch1_e50_TS" #@param {type: 'string'}
if torch.cuda.is_available():
  model.load_state_dict(torch.load(pretrained_model_path))
else:
  model.load_state_dict(torch.load(pretrained_model_path, map_location=torch.device('cpu')))

In [ ]:
#initialize the training script and set number of epochs
model.to(device)
arr = []
models = []
epochs =  50#@param {type: 'number'}
def train():
    criterion = DICELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    num_steps_train = len(train_loader)
    
    print(num_steps_train)
    
    for epoch in range(epochs): #set to previous number of epochs
        t0 = time.time()
        for i, (images, masks) in enumerate(train_loader):
            images = images.to(device)
            masks = masks.to(device)
            outputs = model(images)
            # outputs.to(device)
            loss = criterion(outputs, masks)
            arr.append(loss)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            print('\r Epoch {} of {}   {:.2f}% - loss: {:.4f} - ({:.2f}s)'
                       .format(epoch + 1, epochs, i * 100 / num_steps_train, float
                               (loss), time.time() - t0))
        models.append(model)

In [ ]:
#train the convolutional neural network
train() 

#Save the Model and Create Automated Segmentations for the Training Set

In [ ]:
#name and save the model (JUST CHANGE model_name)
save_path = "/content/gdrive/MyDrive/BIOENG 1340 - Final Project/code/models/" #@param {type: 'string'}
model_name = "Unet_Batch1_e50_TS_32" #@param {type: 'string'} 

if debug_flag:
  save_path = os.path.join(save_path,'debug')

model_dir = os.path.join(save_path, model_name)
os.makedirs(model_dir, exist_ok=True)
torch.save(model.state_dict(), os.path.join(model_dir,model_name))

In [ ]:
#save the automatic segmentations to a folder and list the number of epochs in the title
dices = {}
folder_auto_segmentations = "Unet_Seg_Batch1_Training_32" #@param {type: 'string'}
out = os.path.join(model_dir, folder_auto_segmentations)
total_epochs_run = "50" #@param {type: 'string'}

if not os.path.isdir(out):
  os.mkdir(out)

masked = root_segmented 
originals = root_path
criterion = DICELoss()
for a_file, m in zip(sorted(os.listdir(originals)),sorted(os.listdir(masked))):
  # Read the ground truth mask and convert it to float32
  a_mask = sitk.ReadImage(os.path.join(masked,m))
  a_mask = sitk.GetArrayFromImage(a_mask)
  a_mask = np.float32(a_mask)
  dimensions = a_mask.shape
  torch_shape = (1,1) + dimensions
  a_mask = torch.reshape(torch.from_numpy(a_mask).type(torch.float32), shape=torch_shape).to(device)

  # Read the ground truth image and convert it to a float32
  image = (sitk.ReadImage(os.path.join(originals, a_file)))
  image = sitk.GetArrayFromImage(image)
  dimensions = image.shape
  torch_shape = (1,1) + dimensions
  image = torch.reshape(torch.from_numpy(normalize(image)).type(torch.float32), shape=torch_shape).to(device)

  # Calculate the dice loss
  loss = criterion(model.forward(image), a_mask)
  dices[a_file] = float(loss.cpu().detach().numpy())

  #for saving the output image
  test_output = model.forward(image) #run this
  numpyOutputMatrix = test_output.detach().cpu().numpy()#run this

  # Write the segementation out
  numpyOutputMatrix = numpyOutputMatrix.squeeze(axis=(0,1)) #drop first two axis
  image_out = sitk.GetImageFromArray(numpyOutputMatrix)
  origImage = sitk.ReadImage(os.path.join(originals, a_file))
  image_out.SetOrigin(origImage.GetOrigin())
  image_out.SetSpacing(origImage.GetSpacing())
  image_out.SetDirection(origImage.GetDirection())
  #image.cuda() for float tensors error
  
  #need to copy the information from the original image to enable upsampling if desired
  image_out.CopyInformation(origImage)
  sitk.WriteImage(image_out, os.path.join(out, m.replace(".nii.gz","_auto_seg_Batch1_e")+str(total_epochs_run)+".nii.gz")) #change this each run first and epochs #run this

#Save Dice Loss Info

In [ ]:
#save the dice loss to json
import json
with open('dices_normal.json', 'w') as fp:
    json.dump(dices, fp)
dices

{'BraTS20_Training_007_flair32.nii.gz': 0.08796977996826172,
 'BraTS20_Training_015_flair32.nii.gz': 0.062075018882751465,
 'BraTS20_Training_022_flair32.nii.gz': 0.07609337568283081,
 'BraTS20_Training_033_flair32.nii.gz': 0.057393431663513184,
 'BraTS20_Training_036_flair32.nii.gz': 0.07367700338363647,
 'BraTS20_Training_042_flair32.nii.gz': 0.05533099174499512,
 'BraTS20_Training_049_flair32.nii.gz': 0.08066624402999878,
 'BraTS20_Training_052_flair32.nii.gz': 0.14844805002212524,
 'BraTS20_Training_053_flair32.nii.gz': 0.07026851177215576,
 'BraTS20_Training_054_flair32.nii.gz': 0.12005460262298584,
 'BraTS20_Training_059_flair32.nii.gz': 0.15004563331604004,
 'BraTS20_Training_065_flair32.nii.gz': 0.12843787670135498,
 'BraTS20_Training_068_flair32.nii.gz': 0.07925474643707275,
 'BraTS20_Training_070_flair32.nii.gz': 0.12851214408874512,
 'BraTS20_Training_071_flair32.nii.gz': 0.0799109935760498,
 'BraTS20_Training_081_flair32.nii.gz': 0.032016098499298096,
 'BraTS20_Training_086

In [ ]:
#save the dice loss information
file_name = "Unet_Batch1_e50_32.txt" #@param {type: 'string'}
f = open(os.path.join(model_dir,file_name),"w")
f.write( str(dices))
f.close()

# Create Automated Segmentations for the Test Set

In [ ]:
# Save the automatic segmentations for the test set
dices = {}
test_segmented = '/content/gdrive/MyDrive/BIOENG 1340 - Final Project/segs/test_segs_WT32' #@param {type: 'string'}
test_path = '/content/gdrive/MyDrive/BIOENG 1340 - Final Project/images/test_images32' #@param {type: 'string'}
model_dir =  '/content/gdrive/MyDrive/BIOENG 1340 - Final Project/code/models/Unet_Batch1_e50_TS_32/' #@param {type: 'string'}
folder_auto_segmentations = "Unet_Seg_Batch1_Test_32" #@param {type: 'string'}
out = os.path.join(model_dir,folder_auto_segmentations)
os.makedirs(out, exist_ok=True)
total_epochs_run = "50" #@param {type: 'string'}

originals = sorted(os.listdir(test_path))
masked = sorted(os.listdir(test_segmented))
print(originals) #Nothing is printed here that's prob the issue
print(masked)
print(test_path)
print(test_segmented)

if debug_flag:
  originals = originals[:3]
  masked = masked[:3]

criterion = DICELoss()
for a_file, m in zip(originals,masked):
  a_mask = sitk.ReadImage(os.path.join(test_segmented,m)) #modified the naming of the _MAIN to _SEG2_Binary to find corresponding files
  a_mask = sitk.GetArrayFromImage(a_mask)
  a_mask = np.float32(a_mask)
  dimensions = a_mask.shape
  torch_shape = (1,1) + dimensions
  a_mask = torch.reshape(torch.from_numpy(a_mask).type(torch.float32), shape=torch_shape).to(device)

  image = sitk.ReadImage(os.path.join(test_path, a_file))
  image = sitk.GetArrayFromImage(image)
  dimensions = image.shape
  torch_shape = (1,1) + dimensions
  image = torch.reshape(torch.from_numpy(normalize(image)).type(torch.float32), shape=torch_shape).to(device)

  loss = criterion(model.forward(image), a_mask)
  dices[a_file] = float(loss.cpu().detach().numpy())

  #for saving the output image
  test_output = model.forward(image) #run this
  numpyOutputMatrix = test_output.detach().cpu().numpy()#run this

  numpyOutputMatrix = numpyOutputMatrix.squeeze(axis=(0,1)) #drop first two axis
  image_out = sitk.GetImageFromArray(numpyOutputMatrix)
  origImage = sitk.ReadImage(os.path.join(test_path, a_file))
  image_out.SetOrigin(origImage.GetOrigin())
  image_out.SetSpacing(origImage.GetSpacing())
  image_out.SetDirection(origImage.GetDirection())
  #image.cuda() fo float tensors error
  #need to copy the information from the original image to enable upsampling if desired
  image_out.CopyInformation(origImage)
  sitk.WriteImage(image_out, os.path.join(out, m.replace(".nii.gz","_auto_seg_Batch1_e")+str(total_epochs_run)+".nii.gz")) #change this each run first and epochs #run this

['BraTS20_Training_001_flair32.nii.gz', 'BraTS20_Training_017_flair32.nii.gz', 'BraTS20_Training_026_flair32.nii.gz', 'BraTS20_Training_031_flair32.nii.gz', 'BraTS20_Training_040_flair32.nii.gz', 'BraTS20_Training_057_flair32.nii.gz', 'BraTS20_Training_058_flair32.nii.gz', 'BraTS20_Training_064_flair32.nii.gz', 'BraTS20_Training_067_flair32.nii.gz', 'BraTS20_Training_073_flair32.nii.gz', 'BraTS20_Training_074_flair32.nii.gz', 'BraTS20_Training_076_flair32.nii.gz', 'BraTS20_Training_079_flair32.nii.gz', 'BraTS20_Training_083_flair32.nii.gz', 'BraTS20_Training_094_flair32.nii.gz', 'BraTS20_Training_095_flair32.nii.gz', 'BraTS20_Training_109_flair32.nii.gz', 'BraTS20_Training_110_flair32.nii.gz', 'BraTS20_Training_111_flair32.nii.gz', 'BraTS20_Training_115_flair32.nii.gz', 'BraTS20_Training_117_flair32.nii.gz', 'BraTS20_Training_120_flair32.nii.gz', 'BraTS20_Training_125_flair32.nii.gz', 'BraTS20_Training_127_flair32.nii.gz', 'BraTS20_Training_141_flair32.nii.gz', 'BraTS20_Training_142_fl

# Apply Binary Threshold to the Training and Test Set

In [ ]:
# Applies binary thresholding to an imageset
def binary_thresh(seg_folder, save_folder):
  #get directory
  print(seg_folder)
  print(os.listdir(seg_folder))
  seg_dir = sorted(os.listdir(seg_folder)) #Error here - it outputs the training instead of the test sets when applied in the second function 
  print(seg_dir)
  #create folder
  if os.path.isdir(save_folder) == False:
    os.mkdir(save_folder)

  # Binary threshold each segmentation and export them
  for seg in seg_dir:
    prediction = sitk.ReadImage(seg_folder + seg)
    prediction = sitk.BinaryThreshold(prediction, lowerThreshold=0, upperThreshold=255, insideValue=1, outsideValue=0)
    sitk.WriteImage(prediction, save_folder + "binary_"  + seg)
    print("Thresholded: " + seg)

In [ ]:
#apply binary thresholding to the training set
binary_thresh("/content/gdrive/MyDrive/BIOENG 1340 - Final Project/code/models/Unet_Batch1_e50_TS_32/Unet_Seg_Batch1_Training_32/", 
            "/content/gdrive/MyDrive/BIOENG 1340 - Final Project/code/models/Unet_Batch1_e50_TS_32/Unet_Seg_Batch1_Training_Binary_32/")

Thresholded: BraTS20_Training_007_seg_WT32_auto_seg_Batch1_e50.nii.gz
Thresholded: BraTS20_Training_015_seg_WT32_auto_seg_Batch1_e50.nii.gz
Thresholded: BraTS20_Training_022_seg_WT32_auto_seg_Batch1_e50.nii.gz
Thresholded: BraTS20_Training_033_seg_WT32_auto_seg_Batch1_e50.nii.gz
Thresholded: BraTS20_Training_036_seg_WT32_auto_seg_Batch1_e50.nii.gz
Thresholded: BraTS20_Training_042_seg_WT32_auto_seg_Batch1_e50.nii.gz
Thresholded: BraTS20_Training_049_seg_WT32_auto_seg_Batch1_e50.nii.gz
Thresholded: BraTS20_Training_052_seg_WT32_auto_seg_Batch1_e50.nii.gz
Thresholded: BraTS20_Training_053_seg_WT32_auto_seg_Batch1_e50.nii.gz
Thresholded: BraTS20_Training_054_seg_WT32_auto_seg_Batch1_e50.nii.gz
Thresholded: BraTS20_Training_059_seg_WT32_auto_seg_Batch1_e50.nii.gz
Thresholded: BraTS20_Training_065_seg_WT32_auto_seg_Batch1_e50.nii.gz
Thresholded: BraTS20_Training_068_seg_WT32_auto_seg_Batch1_e50.nii.gz
Thresholded: BraTS20_Training_070_seg_WT32_auto_seg_Batch1_e50.nii.gz
Thresholded: BraTS20

In [ ]:
#apply binary thresholding to the training set
binary_thresh("/content/gdrive/MyDrive/BIOENG 1340 - Final Project/code/models/Unet_Batch1_e50_TS_32/Unet_Seg_Batch1_Test_32/", 
            "/content/gdrive/MyDrive/BIOENG 1340 - Final Project/code/models/Unet_Batch1_e50_TS_32/Unet_Seg_Batch1_Test_Binary_32/")

/content/gdrive/MyDrive/BIOENG 1340 - Final Project/code/models/Unet_Batch1_e50_TS_32/Unet_Seg_Batch1_Test_32/
['BraTS20_Training_001_seg_WT32_auto_seg_Batch1_e50.nii.gz', 'BraTS20_Training_017_seg_WT32_auto_seg_Batch1_e50.nii.gz', 'BraTS20_Training_026_seg_WT32_auto_seg_Batch1_e50.nii.gz', 'BraTS20_Training_031_seg_WT32_auto_seg_Batch1_e50.nii.gz', 'BraTS20_Training_040_seg_WT32_auto_seg_Batch1_e50.nii.gz', 'BraTS20_Training_057_seg_WT32_auto_seg_Batch1_e50.nii.gz', 'BraTS20_Training_058_seg_WT32_auto_seg_Batch1_e50.nii.gz', 'BraTS20_Training_064_seg_WT32_auto_seg_Batch1_e50.nii.gz', 'BraTS20_Training_067_seg_WT32_auto_seg_Batch1_e50.nii.gz', 'BraTS20_Training_073_seg_WT32_auto_seg_Batch1_e50.nii.gz', 'BraTS20_Training_074_seg_WT32_auto_seg_Batch1_e50.nii.gz', 'BraTS20_Training_076_seg_WT32_auto_seg_Batch1_e50.nii.gz', 'BraTS20_Training_079_seg_WT32_auto_seg_Batch1_e50.nii.gz', 'BraTS20_Training_083_seg_WT32_auto_seg_Batch1_e50.nii.gz', 'BraTS20_Training_094_seg_WT32_auto_seg_Batch1_e